In [1]:
# stl-SSNP-07.py
# 2021-11-25
#
# read dem-all-7.csv file
# transposed array, work for space delimeter
# x,y,z = np.loadtxt(path_data + 'dem_all-7.csv', unpack=True)
#
# run on myBinder.org JupyterLab
# 程式檔 path is '/home/jovyan/STL'
# 資料檔 path is '/home/jovyan/STL/data'
# 圖形檔 path is '/home/jovyan/STL/3Dstl'
#
import os
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
import numpy as np
import pprint
from stl import mesh
#
path_data  = './data/'
path_3Dstl = './3Dstl/'
#
## =======================================================================
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
float_To_int = np.vectorize(np.int32)
#
data = np.loadtxt(path_data + 'dem-all-7.csv',delimiter=',',dtype='float')
min_item = np.amin(data, axis=0) # axis=0 --> min of each column
max_item = np.amax(data, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0]), float_To_int(min_item[1]))
print ('Right Up point: ' , float_To_int(max_item[0]), float_To_int(max_item[1]))
print ('height (min, max)', min_item[2], max_item[2])
print ('\n')
height_min = min_item[2]
height_max = max_item[2]
#
# Left Down point: 0, 0
# Right Up point:  4000, 7800
# height (min, max) 0, 357.21
#
TM2_X = float_To_int(data[..., 0]) # 二度分帶 X座標
TM2_Y = float_To_int(data[..., 1]) # 二度分帶 Y座標
TM2_Z = data[..., 2]               # DEM file z value, float type
#
xy_list = []
for y in np.arange(0, 78.2, 0.2) :
    row_list = []
    for x in np.arange(0, 40.2, 0.2):
        # posi_y = str(np.around(y,1))
        posi_str = '(' + str(np.around(x,1)) + ',' + str(np.around(y,1)) + ')'
        row_list.append(posi_str)
    # eed for
    xy_list.append(row_list)
#end for
#
for row in range(8, -1, -1) :
    final_str = ''
    for col in xy_list[row][190:201] :
        final_str = final_str + col + ' '
    # end for
    # print(final_str)
    # print('\n')
# end for
#
## -------------------------------------------------------------------------
#
# XYlist is sorted (x, y), first order is y, then x
#
XYlist = list(zip(TM2_X,TM2_Y))
# X, Y value
X = np.arange(0, 4020, 20)
Y = np.arange(0, 7820, 20)
Xmesh, Ymesh = np.meshgrid(X, Y)  # x-y 平面的網格
#
# 7800/20 ==> 391
# 4020/20 ==> 201
#
Zmesh = np.zeros(shape=(391,201),dtype=np.float16)
i = 0 ; j = 0
#
for y in np.arange(0, 7820, 20) :
    z = np.zeros(shape=(201),dtype=np.float16)
    zi = 0
    for x in np.arange(0, 4020, 20):
        if (x,y) in XYlist :
            # print(str(zi), list([x,y]), TM2_Z[i])
            z[zi] = TM2_Z[i]
            i = i + 1
        else :
            # print(str(zi))
            z[zi] = 0.0
        # end if
        zi = zi + 1
    # end for
    # pprint.pprint(z)
    Zmesh[j] = z
    j = j + 1
#end for
#
# print ('\n')
# pprint.pprint(Zmesh[-1])
#
factor = 0.01
Xmesh_3D = np.around(factor * Xmesh, 1)
Ymesh_3D = np.around(factor * Ymesh, 1)
Zmesh_3D = np.around(0.015  * Zmesh, 3)
#
XYZ_list = np.stack([Xmesh_3D, Ymesh_3D, Zmesh_3D], axis=2)
#
min_item = np.amin(XYZ_list, axis=0) # axis=0 --> min of each column
max_item = np.amax(XYZ_list, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0]), float_To_int(min_item[1]))
print ('Right Up point: ' , float_To_int(max_item[0]), float_To_int(max_item[1]))
print ('height (min, max)', 0.015*height_min, 0.015*height_max)
print ('\n')
#
# XYZ_list --> shape=(391,201)
# create dem_rows
#
dem_rows = []
for i in range(0,391) :
    dem_rows.append([list(xyz) for xyz in XYZ_list[i]])  
# end for
#
# adjust to not show the hole
for i in (385,386,387,388,389,390) :
    dem_rows[i] = [[xyz[0],xyz[1],0.0 ] for xyz in XYZ_list[i]]
# end for
#
# adjust to not show the hole
for i in (370,371,372,373,374,375,376,377,378,379,380,381,382,383,384) :
    j = 0
    for xyz in XYZ_list[i]:
        if j > 195 :
            dem_rows[i][j] = [xyz[0],xyz[1],0.0]
        # end if
        j = j + 1
    # end for
# end for
#
# Create 312810 triangle faces
tri_faces = np.zeros(312810, dtype=mesh.Mesh.dtype)
#
# each row has  400 faces.
# 391 rows have 391x400 faces  ==> 312800 faces
# plus buttom 5 faces ---> 10 triangle
#
k = 0  # <--- ['vectors'] index
for i in range(0,390,1):
    for j in range(0,200,1):
        tri_faces['vectors'][k  ] = np.array([ dem_rows[i][j],  dem_rows[i][j+1],  dem_rows[i+1][j] ])
        tri_faces['vectors'][k+1] = np.array([ dem_rows[i][j+1],dem_rows[i+1][j+1],dem_rows[i+1][j] ])
        k = k + 2
    # end for
# end for
#
# up layer verteices
A_up = [ 0.0,  0.0, 0.0] # 0
B_up = [40.0,  0.0, 0.0] # 1
C_up = [40.0, 78.0, 0.0] # 2
D_up = [ 0.0, 78.0, 0.0] # 3
#
# down layer verteices
a_dn = [ 0.0,  0.0,-1.0] # 4
b_dn = [40.0,  0.0,-1.0] # 5
c_dn = [40.0, 78.0,-1.0] # 6
d_dn = [ 0.0, 78.0,-1.0] # 7
#
# [0,1,3],[1,2,3] ---> omit top face
#
B_tri = np.array([[[ 0.0, 78.0,-1.0],[40.0, 78.0,-1.0],[ 0.0,  0.0,-1.0]],
                  [[40.0, 78.0,-1.0],[40.0,  0.0,-1.0],[ 0.0,  0.0,-1.0]], 
                  [[ 0.0,  0.0, 0.0],[ 0.0,  0.0,-1.0],[40.0,  0.0,-1.0]],
                  [[40.0,  0.0,-1.0],[40.0,  0.0, 0.0],[ 0.0,  0.0, 0.0]], 
                  [[40.0,  0.0, 0.0],[40.0,  0.0,-1.0],[40.0, 78.0,-1.0]],
                  [[40.0, 78.0,-1.0],[40.0, 78.0, 0.0],[40.0,  0.0, 0.0]], 
                  [[40.0, 78.0,-1.0],[ 0.0, 78.0,-1.0],[ 0.0, 78.0, 0.0]],
                  [[40.0, 78.0,-1.0],[ 0.0, 78.0, 0.0],[40.0, 78.0, 0.0]], 
                  [[ 0.0, 78.0,-1.0],[ 0.0,  0.0,-1.0],[ 0.0, 78.0, 0.0]],
                  [[40.0,  0.0,-1.0],[ 0.0,  0.0, 0.0],[ 0.0, 78.0, 0.0]]])
#
tri_faces['vectors'][312800] = np.array(B_tri[0])
tri_faces['vectors'][312801] = np.array(B_tri[1])
tri_faces['vectors'][312802] = np.array(B_tri[2])
tri_faces['vectors'][312803] = np.array(B_tri[3])
tri_faces['vectors'][312804] = np.array(B_tri[4])
tri_faces['vectors'][312805] = np.array(B_tri[5])
tri_faces['vectors'][312806] = np.array(B_tri[6])
tri_faces['vectors'][312807] = np.array(B_tri[7])
tri_faces['vectors'][312808] = np.array(B_tri[8])
tri_faces['vectors'][312809] = np.array(B_tri[9])
#
# Write the mesh to file "SSNP3D.stl"
SSNP3D = mesh.Mesh(tri_faces)
SSNP3D.save(path_3Dstl + 'SSNP3D.stl')
#


當前工作目錄 ==> /home/jovyan/STL
Left Down point: 0 0
Right Up point:  4000 8500
height (min, max) 0.0 357.21


Left Down point: [0 0 0] [0 0 0]
Right Up point:  [ 0 78  0] [ 0 78  0]
height (min, max) 0.0 5.358149999999999


